In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch/minsearch.py

--2025-06-17 21:50:55--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5820 (5.7K) [text/plain]
Saving to: ‘minsearch.py.1’

minsearch.py.1      100%[===================>]   5.68K  --.-KB/s    in 0s      

2025-06-17 21:50:55 (66.9 MB/s) - ‘minsearch.py.1’ saved [5820/5820]



In [2]:
import requests 

In [139]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [141]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [5]:
import minsearch

In [6]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [7]:
q = "the course has already started, Can I join?"

In [8]:
index.fit(documents)

In [9]:
boost={"question": 2.0, "section": 0.5 }

In [10]:
filter_dict={"course": "data-engineering-zoomcamp"}

In [11]:
results = index.search( query=q,
    filter_dict={},
    boost_dict=boost,
    num_results=5,
    output_ids=False,)

In [12]:
results

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequen

In [13]:
from openai import OpenAI

In [20]:
client=OpenAI()

In [93]:
def search(query):
    boost={"question": 2.0, "section": 0.5 }
    filter_dict={"course": "data-engineering-zoomcamp"}
    
    results = index.search( query=query,
        filter_dict=filter_dict,
        boost_dict=boost,
        num_results=10,
        output_ids=False,)

    return results
    

In [170]:
def build_prompt(query, search_results):
    prompt_template="""
        You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
        Use only the facts from the CONTEXT when answering the QUESTION.
        
        QUESTION: {question}
        
        CONTEXT:
        {context}
        """.strip()

    context=""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()

    return prompt

In [82]:
def llm(prompt):
    response = client.chat.completions.create(model='gpt-4o-mini', messages=[{"role": "user", "content": prompt}])

    return response.choices[0].message.content

In [96]:
query="the course has already started, Can I join?"

def rag(query):
    search_results=search(query)
    prompt=build_prompt(query, search_results)
    answer=llm(prompt)

    return answer

In [97]:
rag(query)

"Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."

In [99]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [100]:
from elasticsearch import Elasticsearch

In [109]:
es_client=Elasticsearch('http://localhost:9200')

In [110]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

In [111]:
index_name="course-questions"

In [112]:
es_client.indices.create(index=index_name, body=index_settings, )

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [113]:
from tqdm.auto import tqdm

In [142]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [167]:
def elastic_search(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    result_docs = []

    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [168]:
query="How do execute a command on a Kubernetes pod?"
elastic_search(query)

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': 'Launch the container im

In [164]:
def elastic_rag(query):
    search_results=elastic_search(query)
    prompt=build_prompt(query, search_results)
    answer=llm(prompt)

    return answer

In [165]:
query="How do copy a file to a Docker container?"

In [166]:
elastic_rag(query)

'You can copy files from your local machine into a Docker container using the `docker cp` command. The basic syntax is as follows:\n\n```\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\n```'

In [169]:
context_template = """
Q: {question}
A: {text}
""".strip()

In [171]:
def build_prompt_tpc(query, search_results):
    prompt_template="""
        You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
        Use only the facts from the CONTEXT when answering the QUESTION.
        
        QUESTION: {question}
        
        CONTEXT:
        {context}
        """.strip()

    context=""

    for doc in search_results:
        context = context + context_template.format(question=doc['question'], text=doc['text']) + "\n\n"
        
    prompt = prompt_template.format(question=query, context=context).strip()

    return prompt

In [178]:
prompt_tpc = build_prompt_tpc(query, elastic_search(query))

In [180]:
len(prompt_tpc)

1342

In [181]:
import tiktoken

In [183]:
encoding = tiktoken.encoding_for_model("gpt-4o-mini")

In [184]:
prompt_tpc_tokens = encoding.encode(prompt_tpc)

In [186]:
len(prompt_tpc_tokens)

300